<div align='center'><font size="10" color="blue"> DEEP LEARNING FOR NLP </font></div>

Cite: <a> https://github.com/duyvuleo/VNTC/tree/master </a>

Aims: Build a simple neural network for text classification and report the training process and results.
- This code to concat all raw data of 10 Topics newspaper from `txt` file into a `dataframe`
- EDA the data and do the transform techiniques to a balanced data
- Normalize data
- Build a fully connection basic model with accuracy `~ 60%`
- Demo with Pytorch Interference

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install nltk vietnam-number pyvi -q

In [7]:
import pandas as pd
import numpy as np
import string
import os
import re
import html
from vietnam_number import n2w_single
from pyvi import ViTokenizer
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from collections import Counter

In [9]:
cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")

if cuda_available:
    print(f"CUDA Device Count: {torch.cuda.device_count()}")
    print(f"Current CUDA Device: {torch.cuda.current_device()}")
    print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Please check your PyTorch installation.")


CUDA Available: False
CUDA is not available. Please check your PyTorch installation.


> Ta có được 2 file `train_data.csv` và `test_data.csv` qua quá trình nối các dữ liệu từ file gốc.

In [15]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/DL for NLP/train_data.csv'
# test_dir = 'https://studenthcmusedu-my.sharepoint.com/:f:/g/personal/21280096_student_hcmus_edu_vn/ErTmqTQYpO1OpzdCyFLw0O8Bj9Pb3hvNdgkz7wshmblHIA?e=qUyJoP'


In [17]:
df = pd.read_csv(train_dir, header=0)
display(df)

,topic,content
0,Chinh tri Xa hoi,Thành lập dự án POLICY phòng chống HIV/AIDS ở...
1,Chinh tri Xa hoi,Hơn 16.000 khách đến vịnh Nha Trang Theo trực...
2,Chinh tri Xa hoi,TPHCM: Khai trương dịch vụ lặn biển săn cá mậ...
3,Chinh tri Xa hoi,Du lịch VN sẽ có tư vấn nước ngoài Ông Phạm T...
4,Chinh tri Xa hoi,Quy chế tuyển sinh 2006: Không làm tròn điểm ...
...,...,...
33754,Vi tinh,Điện thoại di động tương lai trông như thế nào...
33755,Vi tinh,Internet sẽ tăng tốc 1.000 lần \nTrong tương l...
33756,Vi tinh,Phần lớn thế giới thứ 3 thất bại với chính phủ...
33757,Vi tinh,'Doom 3' giành chiến thắng kép \nTrò chơi hành...


### EDA

In [18]:
 df.sample(5)

,topic,content
12880,Phap luat,4 hình thức kỷ luật nhóm sinh viên tham gia đậ...
682,Chinh tri Xa hoi,"Mỹ viện trợ thêm 3,3 triệu USD rà phá mìn (NL..."
27165,The thao,Brazil giao đấu với Argentina ở chung kết?\nTr...
11055,Kinh doanh,Phát triển công nghiệp hỗ trợ: Chìa khóa thu h...
31075,Van hoa,Bức xúc khi đọc về Hà Kiều Anh\nTôi thực sự cả...


In [23]:
df.nunique()

,0
topic,10
content,33711


In [20]:
duplicates = df[df.duplicated(subset=['content'], keep=False)].sort_values(by='content')
print(duplicates[['content', 'topic']])
df = df.drop_duplicates()
df.nunique()

                                                 content             topic
536     5 sai phạm lớn của ông Nguyễn Việt Tiến PMU 1...  Chinh tri Xa hoi
14636   5 sai phạm lớn của ông Nguyễn Việt Tiến PMU 1...         Phap luat
466     Chôn rơm lấy hàng tỉ tiền tiêu hủy gia cầm Ng...  Chinh tri Xa hoi
14464   Chôn rơm lấy hàng tỉ tiền tiêu hủy gia cầm Ng...         Phap luat
600     Cá độ thời internet Một điểm tổ chức cá độ bó...  Chinh tri Xa hoi
...                                                  ...               ...
12217  Điện thoại di động Samsung phát nổ\nHôm qua, t...        Kinh doanh
17503  Đạt cực khoái chỉ sau một cú bấm?\nMột điều kỳ...          Suc khoe
9804   Đạt cực khoái chỉ sau một cú bấm?\nMột điều kỳ...          Khoa hoc
16880  Đầu càng tròn ngáy càng to\nNhững anh chàng đầ...          Suc khoe
8991   Đầu càng tròn ngáy càng to\nNhững anh chàng đầ...          Khoa hoc

[96 rows x 2 columns]


,0
topic,10
content,33711


In [ ]:
label_counts = df['topic'].value_counts()
print("Topic distribution:\n", label_counts)

Topic distribution:
 topic
The thao            5298
Chinh tri Xa hoi    5219
Phap luat           3868
Suc khoe            3384
Doi song            3158
Van hoa             3080
The gioi            2898
Kinh doanh          2552
Vi tinh             2481
Khoa hoc            1820
Name: count, dtype: int64


> Assume that data has balance.

In [24]:
df.count()

,0
topic,33758
content,33758


### Clean text

In [27]:
import requests
import re
import html

def load_proper_names(file_url):
    response = requests.get(file_url)
    if response.status_code == 200:
        return response.text.splitlines()
    else:
        raise Exception(f"Error loading file: {response.status_code}")

def process_content(text, proper_names):
  """
    Đã xử lý được:
    - lowercase toàn bộ text, uppercase những danh từ riêng tiếng Việt
    - gộp các đoạn văn thành 1 dòng
    - bỏ dấu trong câu
    - tokenize
    - bỏ các ký tự \n, html...

    Cần bổ sung:
    - Lemmazation các số la mã IV - 4, IX - 9...
    - Giải nghĩa những từ viết tắt như ĐBSCL - Đồng bằng sông Cửu Long, BV - bệnh viện, NNCĐDC - Nạn nhân chất độc da cam...
    không xét trường hợp đã có chú thích trước đó, ví dụ: Người Lao động (NLĐ).
    - Định dạng ngày tháng năm, ví dụ 18/3 thành mười tám tháng ba, tháng 7 thành tháng bảy
    - Xử lý các chữ số.

  """
    # Chuyển về chữ thường
    text = text.lower()

    # Chuyển hóa tên riêng thành chữ hoa
    for name in proper_names:
        text = re.sub(r'\b' + re.escape(name.lower()) + r'\b', name, text)

    # Gộp đoạn văn thành một dòng
    text = ' '.join(text.split())

    # Bỏ dấu câu và ký tự không cần thiết
    text = re.sub(r'[^\w\s]', '', text)

    # Bỏ ký tự HTML
    text = html.unescape(text)

    # Tokenization using ViTokenizer
    tokens = ViTokenizer.tokenize(text)  # Tokenize and format the output as space-separated tokens

    return tokens

# Hàm chính để xử lý cột content trong dataframe
def clean_dataframe(df, proper_names):
    df['content'] = df['content'].apply(lambda x: process_content(x, proper_names))
    return df

# Đọc file tên riêng từ GitHub
proper_names_file = 'https://raw.githubusercontent.com/winstonleedev/tudien/master/tenrieng.txt'
proper_names = load_proper_names(proper_names_file)

# Áp dụng việc xử lý và tokenize cho 1000 dòng tiếp theo (do đã clean thử với 1000 dòng đầu)
df.iloc[1000:2000] = clean_dataframe(df.iloc[1000:2000], proper_names)
print(df.head(100))


               topic                                            content
0   Chinh tri Xa hoi   Thành lập dự án POLICY phòng chống HIV/AIDS ở...
1   Chinh tri Xa hoi   Hơn 16.000 khách đến vịnh Nha Trang Theo trực...
2   Chinh tri Xa hoi   TPHCM: Khai trương dịch vụ lặn biển săn cá mậ...
3   Chinh tri Xa hoi   Du lịch VN sẽ có tư vấn nước ngoài Ông Phạm T...
4   Chinh tri Xa hoi   Quy chế tuyển sinh 2006: Không làm tròn điểm ...
..               ...                                                ...
95  Chinh tri Xa hoi   Nên bỏ hẳn kỳ thi tuyển vào lớp 10 Trong cuộc...
96  Chinh tri Xa hoi   Thành lập Hội Tư vấn Khoa học - Công nghệ &am...
97  Chinh tri Xa hoi   Cà Mau: Nhiều học sinh vùng nông thôn bỏ học ...
98  Chinh tri Xa hoi   Có nên thu trọn gói một lần? Hiện nay hầu hết...
99  Chinh tri Xa hoi   Thí sinh đạt điểm cao nhất môn Kế toán tại An...

[100 rows x 2 columns]


<ipython-input-27-7b1e1e43f686>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = df['content'].apply(lambda x: process_content(x, proper_names))


In [29]:
df[:2000]

,topic,content
0,Chinh tri Xa hoi,Thành lập dự án POLICY phòng chống HIV/AIDS ở...
1,Chinh tri Xa hoi,Hơn 16.000 khách đến vịnh Nha Trang Theo trực...
2,Chinh tri Xa hoi,TPHCM: Khai trương dịch vụ lặn biển săn cá mậ...
3,Chinh tri Xa hoi,Du lịch VN sẽ có tư vấn nước ngoài Ông Phạm T...
4,Chinh tri Xa hoi,Quy chế tuyển sinh 2006: Không làm tròn điểm ...
...,...,...
1995,Chinh tri Xa hoi,tỉnh gia lai đã có 3 đợt tiếp_nhận 20 người là...
1996,Chinh tri Xa hoi,các cá_nhân có trách_nhiệm kê_khai quyết_toán ...
1997,Chinh tri Xa hoi,theo văn_phòng chính_phủ ngày 253 thủ_tướng ch...
1998,Chinh tri Xa hoi,chiều 263 tổng_bí_thư Nông đức mạnh và đoàn đạ...


> **Nhận xét:** text clean chưa đủ tốt, chưa clean toàn bộ tập train (khoảng hơn 33k dòng). Tuy nhiên do thời gian có hạn nên em sẽ dùng cách "tà đạo" một chút để có thể train model được. Đó là lọc ra 20 dòng đầu tiên của mỗi topic đã được clean.

In [36]:
df_200 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DL for NLP/train_200.csv', header = 0)

for topic in range(0, 11):
    topic_data = df_200[df_200['topic'] == topic]
    df_200 = pd.concat([df_200, topic_data], ignore_index=True)

print(df_200.shape)
label_counts = df_200['topic'].value_counts()
print("Topic distribution:\n", label_counts)
df = df_200

(200, 2)
Topic distribution:
 topic
Chinh tri Xa hoi    20
Doi song            20
Khoa hoc            20
Kinh doanh          20
Phap luat           20
Suc khoe            20
The gioi            20
The thao            20
Van hoa             20
Vi tinh             20
Name: count, dtype: int64


> Sau đó build model với cấu trúc như sau:

```plaintext
+--------------------+
|    Input Text      |
+--------------------+
          |
          v
+--------------------+
|    nn.Embedding    |
|   (vocab_size,     |
|   embed_dim)       |
+--------------------+
          |
          v
+--------------------+
|   Mean Pooling     |
| (Tính trung bình   |
|  các vector từ)    |
+--------------------+
          |
          v
+--------------------+
|     nn.Linear      |
|  (embed_dim, 200)  |
|      (fc1)         |
+--------------------+
          |
          v
+--------------------+
|   ReLU Activation  |
+--------------------+
          |
          v
+--------------------+
|     nn.Linear      |
| (200, num_classes) |
|       (fc2)        |
+--------------------+
          |
          v
+--------------------+
|      Output        |
|   (num_classes)    |
+--------------------+


In [ ]:
# Build model
vocab = Counter()
for text in df['clean_content'].values:
    vocab.update(text)
vocab = {word: i + 1 for i, (word, count) in enumerate(vocab.items())}  # +1 cho chỉ số padding

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['topic'].values)

train_texts, valid_texts, train_labels, valid_labels = train_test_split(df['clean_content'].values, labels, test_size=0.2, random_state=42)

# Tạo tập dữ liệu từ tập huấn luyện và tập xác thực
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenization và chuyển đổi thành chỉ số trong phương thức này
        tokenized_text = [vocab[token] for token in text if token in vocab]
        return torch.tensor(tokenized_text), torch.tensor(label)

train_dataset = TextClassificationDataset(train_texts, train_labels)
valid_dataset = TextClassificationDataset(valid_texts, valid_labels)

# Load data
test_dataset = pd.read_csv('/content/test_dataset.csv', header=0)
test_dataset['clean_content'] = test_dataset['content'].apply(preprocessing)
test_dataset['clean_content'] = test_dataset['clean_content'].apply(word_tokenize)
test_texts = test_dataset['clean_content'].values
test_labels = label_encoder.transform(test_dataset['topic'].values)

test_dataset = TextClassificationDataset(test_texts, test_labels)

# DataLoader
def collate_batch(batch):
    texts, labels = zip(*batch)
    texts_padded = nn.utils.rnn.pad_sequence([text.clone().detach() for text in texts], batch_first=True, padding_value=0)  # Đệm các chuỗi
    return texts_padded, torch.tensor(labels)

train_loader = DataLoader(train_dataset, batch_size=2, collate_fn=collate_batch, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=2, collate_fn=collate_batch, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, collate_fn=collate_batch, shuffle=False)

# Định nghĩa mô hình mạng nơ-ron
class SimpleNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(SimpleNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc1 = nn.Linear(embed_dim, 200)  # Điều chỉnh kích thước đầu vào ở đây
        self.fc2 = nn.Linear(200, num_classes)  # Lớp đầu ra với num_classes nơ-ron

    def forward(self, text):
        embedded = self.embedding(text)       # Kích thước: (batch_size, seq_length, embed_dim)
        # Mean pooling trên các token để giảm độ dimensionality
        embedded = embedded.mean(dim=1)       # Kích thước: (batch_size, embed_dim)
        x = torch.relu(self.fc1(embedded))    # Áp dụng ReLU trên lớp dày đặc đầu tiên
        output = self.fc2(x)                   # Đầu ra từ lớp dày đặc thứ hai
        return output

# Tham số của mô hình
vocab_size = len(vocab) + 1
embed_dim = 128
num_classes = len(set(labels))

# Khởi tạo mô hình
model = SimpleNN(vocab_size, embed_dim, num_classes)

# Bước 5: Hàm mất mát, bộ tối ưu và vòng lặp huấn luyện
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Di chuyển mô hình đến thiết bị (GPU nếu có sẵn)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Huấn luyện mô hình
def train_model(model, train_loader, criterion, optimizer, device, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for texts, labels in train_loader:
            texts, labels = texts.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(texts)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Đánh giá mô hình
def evaluate_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for texts, labels in data_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy


train_model(model, train_loader, criterion, optimizer, device)
print('---'*20)
# Đánh giá mô hình trên tập valid
valid_accuracy = evaluate_model(model, valid_loader, device)
print(f"Validation Accuracy: {valid_accuracy:.2f}")

# Đánh giá mô hình trên tập test_dataset
test_accuracy = evaluate_model(model, test_loader, device)
print(f"Test Accuracy: {test_accuracy:.2f}")

Epoch 1/20, Loss: 2.3715
Epoch 2/20, Loss: 1.8423
Epoch 3/20, Loss: 1.2492
Epoch 4/20, Loss: 0.8273
Epoch 5/20, Loss: 0.5698
Epoch 6/20, Loss: 0.3000
Epoch 7/20, Loss: 0.2165
Epoch 8/20, Loss: 0.1232
Epoch 9/20, Loss: 0.1408
Epoch 10/20, Loss: 0.0789
Epoch 11/20, Loss: 0.0497
Epoch 12/20, Loss: 0.0592
Epoch 13/20, Loss: 0.0110
Epoch 14/20, Loss: 0.0116
Epoch 15/20, Loss: 0.0458
Epoch 16/20, Loss: 0.0745
Epoch 17/20, Loss: 0.1314
Epoch 18/20, Loss: 0.0957
Epoch 19/20, Loss: 0.0209
Epoch 20/20, Loss: 0.0166
------------------------------------------------------------
Validation Accuracy: 0.60
Test Accuracy: 0.59


In [ ]:
# Bước 1: Lưu mô hình đã huấn luyện
def save_model(model, path):
  torch.save(model.state_dict(), path)

save_model(model, '/content/drive/MyDrive/Sudo Code 2024 - Submission/model.pth')

# Bước 2: Tải mô hình đã lưu
def load_model(model, path="model.pth"):
    model.load_state_dict(torch.load(path))
    model.eval()
    print(f"Model loaded from {path}")
    return model

# Bước 3: Xây dựng giao diện để test tác vụ phân loại từ chuỗi nhập
def classify_text(model, vocab, input_text, device):
    # Tokenization của văn bản đầu vào
    tokenized_text = [vocab.get(token, 0) for token in input_text]  # Ánh xạ từ token sang chỉ số, 0 nếu token không tồn tại trong từ điển
    tokenized_text = torch.tensor([tokenized_text]).to(device)      # Chuyển thành tensor và di chuyển đến thiết bị
    tokenized_text = nn.utils.rnn.pad_sequence(tokenized_text, batch_first=True, padding_value=0)  # Đệm để có cùng chiều dài

    # Dự đoán nhãn của văn bản
    with torch.no_grad():
        output = model(tokenized_text)
        _, predicted_label = torch.max(output, dim=1)

    return predicted_label.item()

# Tạo một giao diện để nhập chuỗi từ bàn phím và dự đoán
def interactive_classification(model, vocab, label_encoder, device):
    print("Nhập tối đa 10 chuỗi văn bản để phân loại ('exit' để thoát):")
    for _ in range(10):  # Chỉ cho phép nhập tối đa 10 chuỗi
        input_text = input(">> ")
        if input_text.lower() == 'exit':
            break
        predicted_label = classify_text(model, vocab, input_text, device)

        # Chuyển đổi nhãn dự đoán sang tên nhãn
        predicted_label_name = label_encoder.inverse_transform([predicted_label])

        print(f"Nhãn dự đoán: {predicted_label} - Tên nhãn: {predicted_label_name[0]}")
    print("Kết thúc nhập liệu.")

# Ví dụ mô hình Neural Network đã định nghĩa
class SimpleNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(SimpleNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc1 = nn.Linear(embed_dim, 200)  # Điều chỉnh kích thước đầu vào ở đây
        self.fc2 = nn.Linear(200, num_classes)  # Lớp đầu ra với num_classes nơ-ron

    def forward(self, text):
        embedded = self.embedding(text)       # Kích thước: (batch_size, seq_length, embed_dim)
        # Mean pooling trên các token để giảm độ dimensionality
        embedded = embedded.mean(dim=1)       # Kích thước: (batch_size, embed_dim)
        x = torch.relu(self.fc1(embedded))    # Áp dụng ReLU trên lớp dày đặc đầu tiên
        output = self.fc2(x)                   # Đầu ra từ lớp dày đặc thứ hai
        return output

# Bước 4: Tải mô hình và chạy giao diện
if __name__ == "__main__":
    # Khởi tạo mô hình, giả sử các tham số sau
    vocab_size = len(vocab) + 1  # Từ điển của bạn cần phải có sẵn
    embed_dim = 128
    num_classes = len(set(labels))  # Số lớp của tác vụ phân loại

    model = SimpleNN(vocab_size, embed_dim, num_classes)

    # Tải mô hình đã lưu
    model = load_model(model, path="/content/drive/MyDrive/Sudo Code 2024 - Submission/model.pth")

    # Xác định thiết bị
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Chạy giao diện nhập văn bản từ bàn phím để phân loại
    interactive_classification(model, vocab, label_encoder, device)

<ipython-input-144-d43c6f98158d>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


Model loaded from /content/drive/MyDrive/Sudo Code 2024 - Submission/model.pth
Nhập tối đa 10 chuỗi văn bản để phân loại ('exit' để thoát):
>> Tình dân tộc, nghĩa đồng bào trong tọa đàm "Chung một tấm lòng"
Nhãn dự đoán: 5 - Tên nhãn: Suc khoe
>> Bệnh nhi nhập viện trong tình trạng sốt cao, ngứa toàn thân, mệt mỏi, khó thở, bụng chướng
Nhãn dự đoán: 5 - Tên nhãn: Suc khoe
>> 1.000 học sinh, sinh viên tham gia tập huấn làm nhà lãnh đạo tương lai
Nhãn dự đoán: 1 - Tên nhãn: Doi song
>> Thi đấu bóng đá giao hữu kỷ niệm Ngày Doanh nhân Việt Nam
Nhãn dự đoán: 3 - Tên nhãn: Kinh doanh
>> Curiosity là tàu đổ bộ hoạt động như một robot dạng xe tự hành trên Sao Hỏa. Bắt đầu hành trình săn tìm sự sống ngoài hành tinh năm 2012, nó đã có một loạt phát hiện chấn động, bao gồm các "khối xây dựng sự sống" đầu tiên.
Nhãn dự đoán: 1 - Tên nhãn: Doi song
>> Rượu có thể thúc đẩy một số loại ung thư hoàn toàn không liên quan đến gan hay đường tiêu hóa nói chung.
Nhãn dự đoán: 9 - Tên nhãn: Vi tinh
>> Tập 